In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.1064445972442627
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 20.5205817642977
200000 30.470967137000333
300000 31.060400572230257
400000 30.637285980674314
500000 30.692353497496388
600000 32.5414240545929
700000 30.079535373690828
800000 30.41279868025228
900000 30.477529435518267
1000000 30.04634060262027
1100000 13.130206932506963
1200000 27.03352668631865
1300000 30.457852743370157
1400000 30.477525103458724
1500000 30.02039894168211
1600000 31.119520944205235
1700000 31.314374119538925
1800000 30.75926426237201
1900000 30.824379295495316
2000000 30.777084813304214
2100000 31.267781600958127
2200000 7.945564375366166
2300000 34.50674571845958
2400000 38.046705155800126
2500000 36.772131167629524
2600000 37.84578736884033
2700000 35.91293261730326
2800000 36.585943900435666
2900000 35.81539687106483
3000000 35.543

29400000 30.807228972819914
29500000 29.625661225269027
29600000 31.316785642861834
29700000 31.68354600003609
29800000 27.017087662913845
29900000 16.95214941161829
30000000 29.616876819526052
30100000 31.697464243015745
30200000 30.84288341238873
30300000 20.180100988431313
30400000 22.77194256848845
30500000 29.244105801483666
30600000 29.1818796407529
30700000 31.10783598721022
30800000 26.754836564410105
30900000 29.510573058315604
31000000 26.81493270913139
31100000 25.162290580533476
31200000 12.968420025050003
31300000 25.05518254538226
31400000 29.198287125333135
31500000 26.93342441313501
31600000 27.942958953432214
31700000 29.999760333351297
31800000 27.836987628366977
31900000 31.32179853204244
32000000 12.372132941179185
32100000 30.204471714652744
32200000 31.216759569999795
32300000 27.824796570505224
32400000 30.812154953583125
32500000 11.044371385917726
32600000 28.931259665691126
32700000 29.619187153802514
32800000 27.27255348830756
32900000 29.87953193579468
33000

59100000 10.704842837904415
59200000 25.22804755311377
59300000 24.454132498184993
59400000 24.34269135935287
59500000 25.61673547295375
59600000 25.87636477975789
59700000 11.892900407197121
59800000 28.014428315251436
59900000 28.276592895691522
60000000 29.089573879395882
60100000 27.458394135750563
60200000 28.064321982862474
60300000 27.542999560509347
60400000 27.595432281501726
60500000 27.685564179617796
60600000 15.255265761270767
60700000 21.604090315304408
60800000 25.18110690158862
60900000 25.153486481721412
61000000 25.074102218225548
61100000 23.875679437778466
61200000 12.659941853786766
61300000 22.005599012661957
61400000 25.176490089661698
61500000 24.51443246203208
61600000 24.234379547064243
61700000 25.967039464940623
61800000 15.236827269130309
61900000 24.300695411940552
62000000 28.39661955703075
62100000 27.414587189300413
62200000 29.31633765858846
62300000 27.494508183710856
62400000 29.290652312115803
62500000 28.34672732610661
62600000 29.103555446636662
6

88800000 15.745425042009876
88900000 22.72289971016721
89000000 23.127906859688416
89100000 12.601435342788095
89200000 18.962037381757355
89300000 23.535235658669233
89400000 23.13082734993048
89500000 11.997131841101623
89600000 22.025055297981673
89700000 22.976965086081055
89800000 24.548960673795584
89900000 23.95057795650722
90000000 26.517067557661207
90100000 25.106392548172945
90200000 24.917836353211793
90300000 26.588301406950457
90400000 25.307227879033984
90500000 26.85935692651046
90600000 25.184245385454606
90700000 22.746170783178993
90800000 23.637173119793037
90900000 11.149905111897997
91000000 22.252017851678627
91100000 24.174489882408235
91200000 22.329788269594026
91300000 9.413231934685054
91400000 22.55377970780223
91500000 23.924449615165788
91600000 16.395509640334847
91700000 16.75104491545728
91800000 23.96582774021889
91900000 22.404015281943135
92000000 24.732705390726313
92100000 25.191754930050305
92200000 24.973896975182328
92300000 26.060696246714773


117900000 22.108713905683015
118000000 21.68585786459831
118100000 24.45687183990236
118200000 22.27583983198608
118300000 21.617878394201902
118400000 21.293988728887065
118500000 20.228355869998456
118600000 21.627637125905302
118700000 20.415760358533657
118800000 19.92798245114011
118900000 20.40796851690183
119000000 13.081486733637089
119100000 17.42469895320801
119200000 7.7360474318503325
119300000 14.423318991201215
119400000 17.1859363029316
119500000 19.50839933561346
119600000 21.230208915155657
119700000 20.11948559665868
119800000 19.897242000462324
119900000 21.236104177039667
120000000 19.996167772763595
120100000 23.014354545650967
120200000 22.67088097553616
120300000 22.308607038745656
120400000 22.770852581759033
120500000 19.889512555194802
120600000 20.995267464214983
120700000 20.150945970448586
120800000 19.96217701066087
120900000 20.311211587322944
121000000 19.510553526885634
121100000 19.92061899968307
121200000 8.209769296656448
121300000 13.18591701086132


...

290000000 5.106637273496954
290100000 5.396928968982494
290200000 4.643387744585249
290300000 5.343229149040776
290400000 5.1096629963019735
290500000 5.347884067883653
290600000 5.241567576812303
290700000 4.835192808725678
290800000 3.6211159295094353
290900000 1.1273943653698286
291000000 1.113445718510049
291100000 5.845101982618295
291200000 4.772487438003866
291300000 5.298543784493177
291400000 5.155006141156757
291500000 5.067951057836366
291600000 4.888012268160728
291700000 4.898106866442897
291800000 5.319791202469949
291900000 5.085095505339574
292000000 6.18610237417789
292100000 5.447576551571532
292200000 5.3222566012517305
292300000 5.255862990644547
292400000 4.728102484700578
292500000 5.345652751052121
292600000 5.031590217702668
292700000 5.248916214738787
292800000 4.8162743216552775
292900000 5.341965058372675
293000000 1.1202210891073385
293100000 1.1084908767292958
293200000 3.9675601615539344
293300000 4.781406067381346
293400000 5.030071830303486
293500000 5.2

319000000 3.985697661445359
319100000 3.931145497260952
319200000 3.7584195267520317
319300000 3.548885934584135
319400000 3.7387047238674223
319500000 3.5934768765135443
319600000 3.460955170505865
319700000 3.7901449100943525
319800000 3.8035903625565965
319900000 4.9558274579239745
320000000 5.130297842501676
320100000 4.129495586568099
320200000 3.7902194957500877
320300000 3.8887126080934524
320400000 4.193335927268924
320500000 3.408006229052071
320600000 3.634657098365846
320700000 3.622145305499539
320800000 4.084481140402511
320900000 0.9686350962155957
321000000 0.7123605939961056
321100000 2.5130776822279026
321200000 3.894748773102921
321300000 3.323629967988136
321400000 3.6887425287857267
321500000 4.081559154892137
321600000 3.7708053775187897
321700000 2.9310688067008663
321800000 4.241157554844147
321900000 4.195088726249098
322000000 4.875160301676972
322100000 5.233127130005569
322200000 4.655847465580318
322300000 4.888642311800436
322400000 4.50976511774508
3225000

347700000 1.4964994740912205
347800000 1.622624832601175
347900000 1.4687963461396514
348000000 1.3780693055336661
348100000 1.9793639948747912
348200000 1.3177028442825196
348300000 1.2494620623972026
348400000 1.2986954328845632
348500000 1.5680090891790646
348600000 1.3040112132745656
348700000 1.3734009584782474
348800000 0.5141016386046529
348900000 0.32890698358858156
349000000 0.791340345918997
349100000 1.3586274136273324
349200000 1.2719793554458692
349300000 1.3513988966185664
349400000 1.378039260741544
349500000 1.2093520984490091
349600000 1.4569485186176458
349700000 1.4131722787712993
349800000 1.3953819299818462
349900000 1.4465051946959389
350000000 1.5332183609900594
350100000 1.5925825012916643
350200000 1.3832538858610008
350300000 1.4376988153194088
350400000 1.4921622413298865
350500000 1.4787303052636676
350600000 1.2973236446261407
350700000 1.2866923350122605
350800000 1.2363995982303944
350900000 1.0068399207886638
351000000 0.3241599396985497
351100000 0.3926

375700000 0.7476674904509008
375800000 0.16654063746540942
375900000 0.1666460752955973
376000000 0.1663912091124091
376100000 0.1655046758001685
376200000 0.1788967185715258
376300000 0.16612389805918404
376400000 0.16540251129963596
376500000 0.1756033518767211
376600000 0.21749467364962236
376700000 0.16405961846937733
376800000 0.16500482155619597
376900000 0.16471077671082288
377000000 0.1593257769051969
377100000 0.1621042763630258
377200000 0.16296091929312548
377300000 0.1970675829814732
377400000 0.16391666329255372
377500000 0.16346215993389024
377600000 0.15998561835141975
377700000 0.15692093040325575
377800000 0.15587649231304923
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 1.2729448927110971
378000000 1.2258026519469876
378100000 1.8071633369921978
378200000 1.2789563379894198
378300000 1.0426943923034906
378400000 0.9051680360128092
378500000 0.9319553354072173
378600000 1.5784355781943604
378700000 0.9762377378207624
378800000 0.88566

403700000 0.8793983698047458
403800000 0.7415538199768629
403900000 0.6681637277962115
404000000 0.823489693181473
404100000 0.7771542177761375
404200000 0.769659867299021
404300000 0.7326338422462264
404400000 0.7801433977014224
404500000 0.8373104765001217
404600000 0.6587828232520362
404700000 0.8271658179468022
404800000 0.6635611152762598
404900000 0.7583317776508867
405000000 0.8210345249984159
405100000 0.7718571337207755
405200000 0.7459851381865403
405300000 0.6869219902826919
405400000 0.8709099628259461
405500000 0.6791064599871041
405600000 0.6672077317333167
405700000 0.8103103533489691
405800000 0.796751046615094
405900000 0.707145996212748
406000000 0.6300613111807651
406100000 0.7938176363060693
406200000 0.7397404742792322
406300000 0.6519825933842606
406400000 0.7119394951356927
406500000 0.7649248034153946
406600000 0.7011660747728103
406700000 0.5828437774475634
406800000 0.6524215477517247
406900000 0.7142193762886584
407000000 0.6933947681529581
407100000 0.612434